In [1]:
import pandas as pd
import numpy as np
import re
# from keras.layers import Dense,GaussianNoise,LeakyReLU,Activation
# from keras.models import Sequential
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB,BernoulliNB,MultinomialNB,CategoricalNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [2]:
df=pd.read_csv('centes_marked.csv')

In [4]:
df.columns

Index(['title', 'location', 'country', 'description', 'url bookretreats.com',
       'Is venue, not person'],
      dtype='object')

In [13]:
desc=df['description']
x_train=desc.dropna().values.astype('str')
classes=df['Is venue, not person']
y_train=classes.dropna().values.astype('int')

In [ ]:
NB = GaussianNB()
NB.fit(x_train, y_train)
BNB=BernoulliNB()
BNB.fit(x_train, y_train)

In [ ]:
y_predict = NB.predict(x_test)
y2_predict=BNB.predict(x_test)
print("Accuracy Normal NB: {:.2f}".format(NB.score(x_test, y_test)))
print("Accuracy Bernulli NB: {:.2f}".format(BNB.score(x_test, y_test)))

In [ ]:
rf = RandomForestClassifier()

rf.fit(x_train, y_train)
score=rf.score(x_test,y_test)
print('Random forest accuracy',score)